In [1]:
import requests
import pandas as pd
from anytree import Node, RenderTree
from functions import *
from Reddit_functions import *

In [2]:
with open('Authentication/database_uri.txt', 'r', encoding="utf8") as f:
    uri = f.read()

In [3]:
with open('Authentication/reddit_credentials.txt', 'r', encoding="utf8") as f:
    credentials = f.read()
    
credentials = credentials.split('\n')

In [4]:
# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth(credentials[0], credentials[1])

In [5]:
# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': credentials[2],
        'password': credentials[3]}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

## Get all Reddit Posts

In [7]:
getRedditPosts()

## Get Comments of Individual Posts

In [ ]:
# while the token is valid (~2 hours) we just add headers=headers to our requests
res = requests.get('https://oauth.reddit.com/r/singapore/comments/rmqevj', headers=headers)
# res = requests.get('https://oauth.reddit.com/r/singapore/comments/r08nub', headers=headers)

conversation_dict, tree = createTree(res, 'rmqevj')

In [ ]:
# getLinks('My Profile: https://auth.geeksforgeeks.org/user/Chinmoy%20Lenka/articles in the portal of https://www.geeksforgeeks.org/')

In [ ]:
df = pd.DataFrame.from_dict(conversation_dict)
df['url'] = df['comment'].apply(lambda x: getLinks(x))
df['link_title'] = df['url'].apply(lambda x: getURLfromList(x))

# df[df['id'] == 'hpoc9nq']
df.to_csv('Datasets/reddit_data.csv', index=False)

df.head()

In [ ]:
for pre, fill, node in RenderTree(tree):
    print("%s%s" % (pre, node.name))